In [1]:
import os
import utils
import pandas as pd
import spotipy
import csv

# Extract Playlist Data

In [2]:
def update_playlist(playlist_name):
    '''
    Updates the playlist with the name given: connects to Spotify API and retrieves track list from which
    a csv file will be crafter with Spotify audio features and calculated audio features for each track.
    '''
    
    path = f'output/playlists/{playlist_name}.csv'
    
    old_playlist = {}
    if os.path.exists(path):
        with open(path) as csvFile:
            csvReader = csv.DictReader(csvFile)
            for rows in csvReader:
                track_key = rows['track_key']
                old_playlist[track_key] = rows
    else: 
        old_playlist = None
            
    # query for playlists to find the playlist id of the playlist your looking to update
    token = utils.get_token()
    sp = spotipy.Spotify(auth=token)     

    results = sp.current_user_playlists()
    api_playlists = results['items']
    while results['next']:
        results = sp.next(results)
        api_playlists.extend(results['items'])

    playlist_found = False
    for item in api_playlists:
        if item['name'] == playlist_name:
            playlist_id = item['id']
            playlist_found = True

    if not playlist_found:
        print(f'Unable to find playlist with name {playlist_name}')
        return   

    # update playlist by connecting to Spotify API and pulling tracks
    try:
        print('Connecting to Spotify to update playlist...')
        playlist_dictionary = update_playlist_dictionary(playlist_id, token, old_playlist)
        print(f'Playlist, {playlist_name}, successfully updated!')
    except: 
        print('fail')
    
    playlist_frame = pd.DataFrame.from_dict(playlist_dictionary, orient = 'index')
    playlist_frame.to_csv(path, index = False)

In [3]:
def update_playlist_dictionary(playlist_id, token, old_playlist):
    '''
    Given a playlist in the form of a dictionary from track_key (track_name + '___' + artist_name)
    to track dictionary, updates the dictionary to update the most up to date tracks in the playlist.
    '''
    
    sp = spotipy.Spotify(auth=token)
    playlist = sp.playlist(playlist_id)['tracks']
    items = playlist['items']
    
    while playlist['next']:
        playlist = sp.next(playlist)
        items.extend(playlist['items'])
        
    playlist_tracks = {}
     
    track_counter = 0    
    if old_playlist != None: 
        for item in items:
            # periodically get a new API token as large playlists often have enough songs that
            # API token expires
            if track_counter % 50 == 0:
                token = utils.get_token()
            track_key = item['track']['name'] + '___' + item['track']['artists'][0]['name']
            if track_key in old_playlist.keys():
                track_dict = old_playlist[track_key].copy()
                playlist_tracks[track_key] = track_dict
            else: 
                try:
                    track_dictionary = utils.create_track_dictionary(item['track'], token)
                    track_counter += 1
                    track_dictionary['added_at'] = item['added_at']
                    rich_feats = utils.collect_rich_features(track_dictionary['preview_url'])
                    track_dictionary.update(rich_feats)
                    playlist_tracks[track_key] = track_dictionary
                except:
                    print('fail')
                    pass
    else:
        for item in items:
            # periodically get a new API token as large playlists often have enough songs that API token expires
            if track_counter % 50 == 0:
                token = utils.get_token()
            try:
                track = item['track']
                track_dictionary = utils.create_track_dictionary(track, token)
                track_dictionary['added_at'] = item['added_at']
                rich_feats = utils.collect_rich_features(track_dictionary['preview_url'])
                track_dictionary.update(rich_feats)
                track_name = track['name']
                artist_name = track['artists'][0]['name']
                track_key = track_name + '___' + artist_name 
                playlist_tracks[track_key] = track_dictionary
            except:
                print('fail')
                pass
            
            track_counter += 1

    return playlist_tracks

In [4]:
playlists_to_track = ['Bodzin Disco','Deep dish','The Stick','Jamie Jones Boiler Room Ibiza Villa',
                      'Seinfeld Haul','The AFTER party','The after after party','Things DnB makes me do',
                      'chillin with the bill','Love songs','Molasses Music','Dune Juuz']

In [5]:
for playlist in playlists_to_track:
    update_playlist(playlist_name = playlist)

Connecting to Spotify to update playlist...
Playlist, Bodzin Disco, successfully updated!
Connecting to Spotify to update playlist...
Playlist, Deep dish, successfully updated!
Connecting to Spotify to update playlist...
Playlist, The Stick, successfully updated!
Connecting to Spotify to update playlist...
Playlist, Jamie Jones Boiler Room Ibiza Villa, successfully updated!
Connecting to Spotify to update playlist...
Playlist, Seinfeld Haul, successfully updated!
Connecting to Spotify to update playlist...
Playlist, The AFTER party, successfully updated!
Connecting to Spotify to update playlist...
Playlist, The after after party, successfully updated!
Connecting to Spotify to update playlist...
Playlist, Things DnB makes me do, successfully updated!
Connecting to Spotify to update playlist...
Playlist, chillin with the bill, successfully updated!
Connecting to Spotify to update playlist...
Playlist, Love songs, successfully updated!
Connecting to Spotify to update playlist...
Playlist,